In [1]:
#explore geo data in the /mnt/storage7//jody/tb_data_2022_08_19.csv
#created - 26.08.2022

from re import sub
from unittest import result
from icecream import ic
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import re
import os
import json
import subprocess
import itertools
from statistics import mean
import pandas as pd
import chart_studio.plotly as py
import plotly.offline as po
import plotly.express as px
import plotly.graph_objs as pg
import matplotlib.pyplot as plt
import pycountry_convert
%matplotlib inline
po.init_notebook_mode(connected = True)
from urllib.request import urlopen
import pycountry
from sklearn import preprocessing
# plotly-geo
# geopandas --upgrade
# shapely
# python_version_tuple
# chart_studio

In [2]:
#load and remove nan
csv_ = "/mnt/storage7//jody/tb_data_2022_08_19.csv"
df = pd.read_csv(csv_)
df=df.dropna(subset=['country_code', 'geographic_source', 'drug_resistance_type', 'sublineage']).reset_index(drop=True)

/mnt/storage7/lwang/miniconda3/envs/ml-s7/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning:

Columns (12,15,16,17,19,20,47,48,49,51,53,54,56,57,58,59,62,63) have mixed types.Specify dtype option on import or set low_memory=False.



In [3]:
#converting 2 letter country code to 3 letter
def country_code_conv(two_letter):
    name = pycountry_convert.country_alpha2_to_country_name(two_letter.upper(), cn_name_format="default")
    three_letter = pycountry_convert.country_name_to_country_alpha3(name, cn_name_format="default")

    return three_letter

df['country_3l_code'] = None
for i, e in enumerate(df['country_code']):
    df['country_3l_code'][i] =  country_code_conv(e)

/tmp/ipykernel_108717/2264284554.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# # turning drug resistance type into number
# le = preprocessing.LabelEncoder()
# le.fit([df["drug_resistance_type"]])

# drug_resistance_type = le.transform(df["drug_resistance_type"])
# # le.inverse_transform([0, 1, 2, 3, 4, 5])


In [4]:
df[['country_code', 'geographic_source','sublineage', 'drug_resistance_type']]

,country_code,geographic_source,sublineage,drug_resistance_type
0,pt,Portugal,lineage2.2.1,MDR-TB
1,pt,Portugal,lineage4.3.4.2,MDR-TB
2,pt,Portugal,lineage4.3.3,MDR-TB
3,pt,Portugal,lineage4.3.4.2,MDR-TB
4,pt,Portugal,lineage4.3.4.2,Pre-XDR-TB
...,...,...,...,...
43469,fi,Finland,lineage2.2.1,Sensitive
43470,fi,Finland,lineage4.1.2.1,Sensitive
43471,fi,Finland,lineage4.1.2,Sensitive
43472,fi,Finland,lineage1.2.2.2,Sensitive


In [7]:
# create a column that indicate whether the samle has mixed infection
df['mixed_infection'] = "None"

for i, x in enumerate(df['sublineage']):
    len_ = len(str(x).split(";"))
    if len_ > 1 and len_ < 3:
        df['mixed_infection'][i] = "Mixed_infection"
    else:
        df['mixed_infection'][i] = "Non-mixed_infection"




/tmp/ipykernel_108717/300186783.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_108717/300186783.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
df.groupby(["mixed_infection"]).count()


,wgs_id,mapped_reads,percent_mapped,median_depth,missingness,main_lineage,sublineage,drug_resistance_type,run_accession,sample_accession,...,culture_collection,isolation_source,strain,submitted_host_sex,center_name,host_tax_id,collected_by,isolate,description,country_3l_code
mixed_infection,,,,,,,,,,,,,,,,,,,,,
Mixed_infection,471,471,471,471,471,471,471,471,471,460,...,0,159,111,16,452,156,60,52,454,471
Non-mixed_infection,43003,43003,43003,43003,43003,43003,43003,43003,43003,42074,...,13,13639,7783,2260,41641,13096,8468,7697,41958,43003


In [39]:
df.groupby(["country_3l_code"]).count()


,wgs_id,mapped_reads,percent_mapped,median_depth,missingness,main_lineage,sublineage,drug_resistance_type,run_accession,sample_accession,...,culture_collection,isolation_source,strain,submitted_host_sex,center_name,host_tax_id,collected_by,isolate,description,mixed_infection
country_3l_code,,,,,,,,,,,,,,,,,,,,,
AFG,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,1,1
AGO,4,4,4,4,4,4,4,4,4,4,...,0,0,0,0,4,0,0,0,4,4
AIA,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,1,1
ALB,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,2,0,0,0,2,2
ARG,238,238,238,238,238,238,238,238,238,238,...,0,238,238,0,238,238,0,0,238,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,194,194,194,194,194,194,194,194,194,194,...,0,72,0,0,194,72,72,72,194,194
UZB,285,285,285,285,285,285,285,285,285,285,...,0,0,0,0,285,0,0,0,285,285
VNM,3304,3304,3304,3304,3304,3304,3304,3304,3304,3257,...,0,1606,0,0,2938,1606,1606,1606,3257,3304


In [63]:
df_no_mix = df.loc[df["mixed_infection"]=="Non-mixed_infection"]
df_no_mix = df_no_mix.groupby(["country_3l_code"]).count()
df_no_mix['country_3l_code'] = df_no_mix.index

df_count = df.groupby(["country_3l_code"]).count()
df_count['country_3l_code'] = df_count.index

df_mix = df.loc[df["mixed_infection"]=="Mixed_infection"]
df_mix = df_mix.groupby(["country_3l_code"]).count()
df_mix['country_3l_code'] = df_mix.index

In [65]:
df_count

,wgs_id,mapped_reads,percent_mapped,median_depth,missingness,main_lineage,sublineage,drug_resistance_type,run_accession,sample_accession,...,isolation_source,strain,submitted_host_sex,center_name,host_tax_id,collected_by,isolate,description,mixed_infection,country_3l_code
country_3l_code,,,,,,,,,,,,,,,,,,,,,
AFG,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,0,0,0,1,1,AFG
AGO,4,4,4,4,4,4,4,4,4,4,...,0,0,0,4,0,0,0,4,4,AGO
AIA,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,0,0,0,1,1,AIA
ALB,2,2,2,2,2,2,2,2,2,2,...,0,0,0,2,0,0,0,2,2,ALB
ARG,238,238,238,238,238,238,238,238,238,238,...,238,238,0,238,238,0,0,238,238,ARG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,194,194,194,194,194,194,194,194,194,194,...,72,0,0,194,72,72,72,194,194,USA
UZB,285,285,285,285,285,285,285,285,285,285,...,0,0,0,285,0,0,0,285,285,UZB
VNM,3304,3304,3304,3304,3304,3304,3304,3304,3304,3257,...,1606,0,0,2938,1606,1606,1606,3257,3304,VNM


In [69]:
_ = df_mix[df_mix['country_3l_code']=="BRA"]["mixed_infection"]
int(_)

8

In [51]:
list_ = []
for i, x in enumerate(df_count["country_3l_code"]):
    if x in df_mix["country_3l_code"]:
        mix = df_mix[df_mix['country_3l_code']==x]["mixed_infection"]
        all = df_count[df_count['country_3l_code']==x]["mixed_infection"]
        ratio = int(mix) / in(all)
        list_.append(ratio):
    else:
        list_.append(0)


    


AFG
AGO
AIA
ALB
ARG
AUS
AUT
AZE
BEL
BGD
BGR
BIH
BLR
BOL
BRA
CAN
CHE
CHL
CHN
CIV
CMR
COD
COG
COL
CPV
CZE
DEU
DJI
DNK
DOM
DZA
ERI
ESP
EST
ETH
FIN
FRA
GAB
GBR
GHA
GIN
GMB
GNB
GNQ
GRC
GRL
GTM
HUN
IDN
IND
IRL
IRN
IRQ
ITA
JPN
KAZ
KEN
KGZ
KIR
KOR
LBN
LBY
LKA
MAR
MDA
MEX
MKD
MLI
MMR
MOZ
MWI
MYS
NGA
NIC
NLD
NPL
PAK
PER
PHL
PNG
POL
PRT
ROU
RUS
RWA
SAU
SDN
SEN
SGS
SLE
SLV
SOM
SRB
SVN
SWE
SWZ
SYC
TGO
THA
TJK
TKM
TLS
TUN
TUR
TZA
UGA
UKR
USA
UZB
VNM
ZAF
ZWE


In [18]:
df.groupby(['country_3l_code']).count()["mixed_infection"]=="mixed_infection"

country_3l_code
AFG    False
AGO    False
AIA    False
ALB    False
ARG    False
       ...  
USA    False
UZB    False
VNM    False
ZAF    False
ZWE    False
Name: mixed_infection, Length: 112, dtype: bool

In [ ]:
fig = px.choropleth(df, color='mixed_infection',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(df, color='drug_resistance_type',
                    locations='country_3l_code',
                   )
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#graphing the geo data
data = dict(type='choropleth', 
            locations = df['country_3l_code'] , 
            # z = df['drug_resistance_type_label'], 
            featureidkey = df['drug_resistance_type'],
            # marker = df["mixed_infection"],
            text = (df['geographic_source'],df['drug_resistance_type']))


layout = dict(title = 'TB-resistance',
            
              geo = dict(showframe = True,
                        projection = {'type':'natural earth'},
                         showlakes = False, 
                         lakecolor = 'rgb(0,191,255)'))


x = pg.Figure(data = [data] ,
layout = layout)
po.iplot(x)

